In [91]:

import pandas as pd
import cv2
import numpy as np



class driveFile:
  def __init__(self):
    self.path=""
  def dataload(self,folder_path):
    img=[]
    label=[]
    import os
    self.path1=os.path.join(self.path,folder_path)
    
    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    img_array=np.array(img)
    return img_array

In [92]:

drive=driveFile()
images,labels=drive.dataload('train_set')

Skipping invalid path or .DS_Store: train_set/FU-athlete-foot
Skipping invalid path or .DS_Store: train_set/BA- cellulitis


In [93]:
images=np.array([img.flatten() for img in images])
labels=np.array(labels)
j=0
label_codes=dict()
for i in np.unique(labels):
    label_codes[i]=j
    j+=1
label_codes
labels=np.array([label_codes[i] for i in labels])
del i,j

In [94]:
class Layer:
    def __init__(self):
        self.input=None
        self.output=None
    def forward(self,input):
        # to be overridden 
        pass 
    def backward(self,output_gradient,learning_rate):
        pass
    

In [95]:
class Dense(Layer):
    def __init__(self,input_size,output_size):
        self.weights=np.random.randn(output_size,input_size)*0.01
        self.bias=np.random.randn(output_size,1)
    def forward(self,input):
        self.input=input
        return np.dot(self.weights,self.input)+self.bias
    def backward(self,output_gradient,learning_rate):
        w_gradient = np.dot(output_gradient, self.input.T)  # (output_size, input_size)
        b_gradient = np.sum(output_gradient, axis=1, keepdims=True)  # (output_size, 1)
        input_gradient = np.dot(self.weights.T, output_gradient)  # (input_size, batch_size)
        self.weights-=learning_rate * w_gradient
        self.bias-=learning_rate * b_gradient
        return input_gradient

In [96]:
class Activation(Layer):
    def __init__(self,activation,activatn_derivative):
        #activation is a variable pointing to a activation method,for ex sigmoid
        self.activation=activation
        self.activatn_derivative=activatn_derivative
    def forward(self,input):
        self.input=input
        return self.activation(self.input)
    def backward(self,output_gradient,learning_rate):
        return np.multiply(output_gradient,self.activatn_derivative(self.input))

In [97]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1/(1+np.exp(-x))
        def sigmoid_derivative(x):
            s=sigmoid(x)
            return s*(1-s)
        super().__init__(sigmoid,sigmoid_derivative)
        

In [98]:
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return (exp(x)-exp(-x))/(exp(x)+exp(-x))
        def tanh_derivative(x):
            t=tanh(x)
            return 1-(t**2)
        super().__init__(tanh,tanh_derivative)



In [99]:

class Softmax(Layer):
    def __init__(self):
        pass
    def forward(self,input):
        return np.exp(input)/np.sum(np.exp(input))
    def backward():
        pass
        

In [100]:
class Loss(Layer):
    def __init__(self,loss_fn,loss_fn_Derivative):
        self.loss_fn=loss_fn
        self.loss_fn_Derivative=loss_fn_Derivative
    def forward(self,y_pred,y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return self.loss_fn(y_pred, y_true)
    def backward(self,y_pred,y_true):
        return self.loss_fn_Derivative(y_pred,y_true)

In [101]:
class BinaryCrossEntropy(Loss):
    def __init__(self):
        def bce(y_pred,y_true):
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)  # Prevent extreme values
            return -np.mean((y_true*np.log(y_pred))+((1-y_true)*np.log(1-y_pred)))
        def bce_derivative(y_pred,y_true):
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)  # Avoid division by zero
            return -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))
        super().__init__(bce,bce_derivative)


In [102]:
class SparseCategoricalCE(Loss):
    def __init__(self):
        def sce(logits,true_labels):
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True)) 
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            true_class_probs = probabilities[np.arange(len(true_labels)), true_labels]
            log_loss = -np.log(true_class_probs + 1e-15)  # Adding a small constant for numerical stability
            return np.mean(log_loss)
        def sce_derivative(logits, true_labels):
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Stability trick
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            batch_size = logits.shape[0]
            probabilities[np.arange(batch_size), true_labels] -= 1
            gradient = probabilities / batch_size
            
            return gradient
        super().__init__(sce,sce_derivative)
    

In [103]:
"""class Model:
    def __init__(self, layers, learning_rate=0.01, batch_size=32, epochs=10):
        self.layers = layers
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.loss_fn = SparseCategoricalCE()

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, loss_gradient):
        for layer in reversed(self.layers):
            loss_gradient = layer.backward(loss_gradient, self.learning_rate)

    def train(self, X, y):
        for epoch in range(self.epochs):
            num_samples = X.shape[0]
            index=np.arange(num_samples)
            np.random.shuffle(index)
            X=X[index]
            y=y[index]
            for i in range(0, num_samples, self.batch_size):
                if(i+self.batch_size==num_samples):
                    X_batch = X[i:num_samples]
                    y_batch = y[i:num_samples]
                else:
                    X_batch = X[i:i + self.batch_size]
                    y_batch = y[i:i + self.batch_size]

                # Forward pass
                predictions = self.forward(X_batch.T)
                
                # Compute loss
                loss = self.loss_fn.forward(predictions, y_batch.T)
                print(f"Epoch {epoch + 1}, Batch {i // self.batch_size + 1}, Loss: {loss}")

                # Backward pass
                loss_gradient = self.loss_fn.backward(predictions, y_batch.T)
                self.backward(loss_gradient)

    def predict(self, X):
        predictions = self.forward(X.T)
        return (predictions >= 0.5).astype(int).flatten()

    def evaluate(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        return accuracy

"""

'class Model:\n    def __init__(self, layers, learning_rate=0.01, batch_size=32, epochs=10):\n        self.layers = layers\n        self.learning_rate = learning_rate\n        self.batch_size = batch_size\n        self.epochs = epochs\n        self.loss_fn = SparseCategoricalCE()\n\n    def forward(self, X):\n        for layer in self.layers:\n            X = layer.forward(X)\n        return X\n\n    def backward(self, loss_gradient):\n        for layer in reversed(self.layers):\n            loss_gradient = layer.backward(loss_gradient, self.learning_rate)\n\n    def train(self, X, y):\n        for epoch in range(self.epochs):\n            num_samples = X.shape[0]\n            index=np.arange(num_samples)\n            np.random.shuffle(index)\n            X=X[index]\n            y=y[index]\n            for i in range(0, num_samples, self.batch_size):\n                if(i+self.batch_size==num_samples):\n                    X_batch = X[i:num_samples]\n                    y_batch = y[i:n

In [104]:

class Model:
    def train(self,images,labels,layers,hyperpara):
        
        for key, value in hyperpara.items():
            setattr(self, key, value)
        self.loss_fn=SparseCategoricalCE()
        self.X=images
        self.y=labels
        self.layers=layers
        self.softmax=Softmax()
        
        for epoch in range(self.epochs):
            num_samples = self.X.shape[0]
            index=np.arange(num_samples)
            np.random.shuffle(index)
            self.X=self.X[index]
            self.y=self.y[index]
            for i in range(0, num_samples, self.batch_size):
                if(i+self.batch_size==num_samples):
                    X_batch = self.X[i:num_samples]
                    y_batch = self.y[i:num_samples]
                else:
                    X_batch = self.X[i:i + self.batch_size]
                    y_batch = self.y[i:i + self.batch_size]
                ActiPredicted=self.forward(X_batch)
                pred_y=self.softmax.forward(ActiPredicted)
                
                loss=self.loss_fn.forward(pred_y,y_batch)
                dloss=self.loss_fn.backward(pred_y,y_batch)
                dsoft=self.softmax.forward(ActiPredicted)
                
                print(f"Epoch {epoch + 1}, Batch {i // self.batch_size + 1}, Loss: {loss}")

                # Backward pass
                
                self.backward(loss_gradient)
                
                
                    
                
    def forward(self,X):
        for layer in self.layers:
            # here layer is obj to class not class*
            X=layer.forward(X)
        return X
        

    def backward(self, loss_gradient):
        for layer in reversed(self.layers):
            loss_gradient = layer.backward(loss_gradient, self.learning_rate)
    def predict(self, X):
        predictions = self.forward(X.T)
        return (predictions >= 0.5).astype(int).flatten()

    def evaluate(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        return accuracy

        

In [105]:
hyperparameters={
    "hiddenLayers":2,
    "learning_rate":0.01,
    "batch_size":25,
    "epochs":15,
    "ActivationFns":None,
    "neurons":None
}
# change
hyperparameters["neurons"]=[2,1]
hyperparameters["ActivationFns"]=['Sigmoid()','Tanh()']
layers=[]
inputsize=images.shape[1]
for i in range(hyperparameters["hiddenLayers"]):
    outputsize=hyperparameters["neurons"][i]
    layers.extend([Dense(inputsize,outputsize),hyperparameters["ActivationFns"][i]])
    inputsize=outputsize


model=Model()
model.train(images,labels,layers,hyperparameters)

ValueError: shapes (2,400) and (25,400) not aligned: 400 (dim 1) != 25 (dim 0)